In [2]:
import numpy as np
from PIL import Image
import pandas
import os
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from keras.utils import to_categorical
import datetime


Using TensorFlow backend.


In [3]:
#Set up dataframe
DATA_DIR = "/Users/alex/Data/CactusIdentification/"
TRAIN_DIR = DATA_DIR + "train/"
LBLS_PATH = DATA_DIR + "train.csv"
print("LOADING DATA: ", datetime.datetime.now())
ims = []
ids = []
for i, filename in enumerate(os.listdir(TRAIN_DIR)):
    np_im = np.asarray(Image.open(os.path.join(TRAIN_DIR, filename))).flatten()
    ims.append(np_im)
    ids.append(filename)
df = pandas.DataFrame(np.array(ims), index=np.array(ids))
print("DATA LOADED ", datetime.datetime.now())

LOADING DATA:  2019-03-15 12:56:30.950577


DATA LOADED  2019-03-15 12:56:37.918413


In [4]:
#Load in training labels to df
lbls = pandas.read_csv(LBLS_PATH, index_col="id")
df = df.join(lbls["has_cactus"])

In [25]:
#Prepare for training/testing
X = df.loc[:, df.columns != "has_cactus"].to_numpy().reshape((17500, 32, 32, 3))
y = df.loc[:, df.columns == "has_cactus"].to_numpy().ravel()
print(X.shape)
print(y.shape)
X_trn, X_tst, y_trn, y_tst = train_test_split(X,y,test_size=0.33)
y_trn = to_categorical(y_trn)
y_tst = to_categorical(y_tst)
print(y_trn.shape)

(17500, 32, 32, 3)
(17500,)


(11725, 2)


In [33]:
#Build model
model = Sequential()
layers = [
    Conv2D(128, kernel_size=3, activation='relu', input_shape=(32, 32, 3), padding='same'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(64, kernel_size=3, activation='relu', padding='same'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(32, kernel_size=3, activation='relu', padding='same'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(2, activation='softmax')
]
for layer in layers:
    model.add(layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
model.fit(X_trn, y_trn, validation_data=(X_tst, y_tst), epochs=3)

Train on 11725 samples, validate on 5775 samples
Epoch 1/3


   32/11725 [..............................] - ETA: 8:26 - loss: 1.4697 - acc: 0.8125

   64/11725 [..............................] - ETA: 4:47 - loss: 2.1876 - acc: 0.5781

   96/11725 [..............................] - ETA: 3:32 - loss: 2.0235 - acc: 0.6458

  128/11725 [..............................] - ETA: 2:55 - loss: 1.6826 - acc: 0.6484

...

11616/11725 [============================>.] - ETA: 0s - loss: 0.0967 - acc: 0.9642

11648/11725 [============================>.] - ETA: 0s - loss: 0.0968 - acc: 0.9640

11680/11725 [============================>.] - ETA: 0s - loss: 0.0973 - acc: 0.9638

11712/11725 [============================>.] - ETA: 0s - loss: 0.0972 - acc: 0.9639

11725/11725 [==============================] - 70s 6ms/step - loss: 0.0972 - acc: 0.9639 - val_loss: 0.0796 - val_acc: 0.9730


In [35]:
#Classify and measure accuracy
score, acc = model.evaluate(X_tst, y_tst)
print("ACCURACY: ", acc)
print("SCORE: ", score)
print(datetime.datetime.now())

  32/5775 [..............................] - ETA: 9s

  64/5775 [..............................] - ETA: 10s

  96/5775 [..............................] - ETA: 11s

 128/5775 [..............................] - ETA: 10s

 160/5775 [..............................] - ETA: 10s

...

5696/5775 [============================>.] - ETA: 0s

5728/5775 [============================>.] - ETA: 0s

5760/5775 [============================>.] - ETA: 0s

5775/5775 [==============================] - 9s 2ms/step


ACCURACY:  0.972987012987013
SCORE:  0.07962013948982928
2019-03-15 16:21:43.447803
